# Requerimiento de Capital Mensual

Calculo del SCR basado en la distribucion de perdida agregada total.

In [41]:
# Cargar librerias y datos
library(data.table)
library(knitr)

# Cargar distribucion de perdida agregada total
perdida_total <- fread("../data/output/perdida_agregada_total_portafolio.csv")

cat("Datos cargados:", nrow(perdida_total), "observaciones")
cat("\nRango:", min(perdida_total$x), "a", max(perdida_total$x))

Datos cargados: 200001 observaciones
Rango: 0 a 2e+09

In [42]:
# Calcular perdida esperada
perdida_esperada <- sum(perdida_total$x * perdida_total$pmf)

cat("Perdida Esperada: $", format(perdida_esperada, big.mark=",", scientific=FALSE))

Perdida Esperada: $ 12,719,851

In [43]:
# Funcion para calcular VaR
calcular_var <- function(datos, alpha) {
  indice <- which(datos$cdf >= alpha)[1]
  return(datos$x[indice])
}

# Calcular VaR para diferentes niveles
niveles <- c(0.90, 0.95, 0.99, 0.995, 0.999)
var_valores <- sapply(niveles, function(alpha) calcular_var(perdida_total, alpha))

# Calcular SCR para cada nivel (SCR = VaR - Perdida Esperada)
scr_valores <- var_valores - perdida_esperada

# Crear tabla de resultados
tabla_scr <- data.frame(
  Nivel_Confianza = c("90%", "95%", "99%", "99.5%", "99.9%"),
  VaR = paste0("$", format(var_valores, big.mark=",", scientific=FALSE)),
  SCR = paste0("$", format(scr_valores, big.mark=",", scientific=FALSE)),
  SCR_Porcentaje = paste0(round((scr_valores/perdida_esperada)*100, 1), "%")
)

kable(tabla_scr, caption = "Solvency Capital Requirement (SCR) por Nivel de Confianza")

cat("\n\nFormula: SCR = VaR - Perdida Esperada")



Table: Solvency Capital Requirement (SCR) por Nivel de Confianza

|Nivel_Confianza |VaR         |SCR         |SCR_Porcentaje |
|:---------------|:-----------|:-----------|:--------------|
|90%             |$24,560,000 |$11,840,149 |93.1%          |
|95%             |$30,830,000 |$18,110,149 |142.4%         |
|99%             |$52,620,000 |$39,900,149 |313.7%         |
|99.5%           |$64,850,000 |$52,130,149 |409.8%         |
|99.9%           |$94,070,000 |$81,350,149 |639.6%         |



Formula: SCR = VaR - Perdida Esperada

In [44]:
# Respuesta final - SCR al 99.5% (estandar regulatorio)
scr_995 <- scr_valores[4]  # 99.5% es el cuarto elemento

cat("======================================================")
cat("\n       REQUERIMIENTO DE CAPITAL MENSUAL")
cat("\n======================================================")
cat("\n")
cat("\nSegun Solvencia II (99.5% confianza):")
cat("\n")
cat("\nSCR = $", format(scr_995, big.mark=",", scientific=FALSE))
cat("\n")
cat("\n======================================================")

       REQUERIMIENTO DE CAPITAL MENSUAL

Segun Solvencia II (99.5% confianza):

SCR = $ 52,130,149
